# News Headlines Sentiment

Use the news api to pull the latest news articles for bitcoin and ethereum and create a DataFrame of sentiment scores for each coin. 

Use descriptive statistics to answer the following questions:
1. Which coin had the highest mean positive score?
2. Which coin had the highest negative score?
3. Which coin had the highest positive score?

In [ ]:
# Initial imports
import os
import pandas as pd
from dotenv import load_dotenv
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from newsapi.newsapi_client import NewsApiClient
from datetime import datetime, timedelta

analyzer = SentimentIntensityAnalyzer()

%matplotlib inline

In [ ]:
# Read your api key environment variable
load_dotenv()

In [ ]:
# Create a newsapi client
newsapi = NewsApiClient(api_key=os.environ["NEWSAPI"])

In [ ]:
# date var to keep date on 10/20/2020 for answers to be consistent, previoiusly used "from_param=datetime.today()"
date = pd.Timestamp("2020-10-20").utcoffset()

In [ ]:
# Fetch the Bitcoin news articles
bitcoin_headlines = newsapi.get_everything(q="bitcoin",
                                           from_param=date,
                                           language='en',
                                           sort_by='relevancy',
                                           page=1)


In [ ]:
# Fetch the Ethereum news articles
ethereum_headlines = newsapi.get_everything(q="ethereum",
                                           from_param=date,
                                           language='en',
                                           sort_by='relevancy',
                                           page=1)


In [ ]:
# Create the Bitcoin sentiment scores DataFrame
bitcoin_sentiments = []

for article in bitcoin_headlines["articles"]:
    try:
        sentiment = analyzer.polarity_scores(article['title'])
      
        bitcoin_sentiments.append({
            "Text": article["title"],
            "Compound": sentiment["compound"],
            "Positive": sentiment["pos"],
            "Negative": sentiment["neg"],
            "Neutral": sentiment["neu"]
            
        })
        
    except AttributeError:
        pass
    
# Create DataFrame
btc_df = pd.DataFrame(bitcoin_sentiments)

# Reorder DataFrame columns
cols =["Compound", "Negative", "Neutral", "Positive", "Text"]
btc_df = btc_df[cols]

btc_df.head()

In [ ]:
# Create the ethereum sentiment scores DataFrame
ethereum_sentiments = []

for article in ethereum_headlines["articles"]:
    try:
        sentiment = analyzer.polarity_scores(article['title'])
      
        ethereum_sentiments.append({
            "Text": article["title"],
            "Compound": sentiment["compound"],
            "Positive": sentiment["pos"],
            "Negative": sentiment["neg"],
            "Neutral": sentiment["neu"]
            
        })
        
    except AttributeError:
        pass
    
# Create DataFrame
eth_df = pd.DataFrame(ethereum_sentiments)

# Reorder DataFrame columns
cols =["Compound", "Negative", "Neutral", "Positive", "Text"]
eth_df = eth_df[cols]

eth_df.head()

In [ ]:
# Describe the Bitcoin Sentiment
btc_df.describe()

In [ ]:
# Describe the Ethereum Sentiment
eth_df.describe()

### Questions:

Q: Which coin had the highest mean positive score on 10/20/20?

A: Bitcoin

Q: Which coin had the highest compound score on 10/20/20?

A: Ethereum

Q. Which coin had the highest positive score on 10/20/20?

A: Bitcoin

---

# Tokenizer

In this section, you will use NLTK and Python to tokenize the text for each coin. Be sure to:
1. Lowercase each word
2. Remove Punctuation
3. Remove Stopwords

In [ ]:
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, PorterStemmer
from string import punctuation
import re

In [ ]:
# Expand the default stopwords list if necessary
expanded_stopwords = ['btc', 'bitcoin', 'eth', 'ethereum', "'", ',', '$', '(', ')', '.']

In [ ]:
# Complete the tokenizer function
def tokenizer(text):
    """Tokenizes text."""
    # Create a list of the words
    text = word_tokenize(text)
    # Convert the words to lowercase
    text = [word.lower() for word in text]
    # Remove the punctuation
    regex = re.compile("[^a-zA-Z]")
    # Remove the stop words
    sw = set(stopwords.words('english') + expanded_stopwords)
    # Lemmatize Words into root words
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in text]
    tokens = [word for word in text if word not in sw]
    return tokens


In [ ]:
# Create a new tokens column for bitcoin
btc_tokens=[]
[btc_tokens.append(tokenizer(text)) for text in btc_df.Text]   
btc_df['Tokens'] = btc_tokens
btc_df.head()

In [ ]:
# Create a new tokens column for ethereum
eth_tokens=[]
[eth_tokens.append(tokenizer(text)) for text in eth_df.Text]   
eth_df['Tokens'] = eth_tokens
eth_df.head()

---

# NGrams and Frequency Analysis

In this section you will look at the ngrams and word frequency for each coin. 

1. Use NLTK to produce the n-grams for N = 2. 
2. List the top 10 words for each coin. 

In [ ]:
from collections import Counter
from nltk import ngrams

In [ ]:
# Generate the Bitcoin N-grams where N=2
btc_bigram_counts = [Counter(ngrams(tokens, n=2)) for tokens in btc_df.Tokens]
dict(btc_bigram_counts[0].most_common(10))

In [ ]:
# Generate the Ethereum N-grams where N=2
eth_bigram_counts = [Counter(ngrams(tokens, n=2)) for tokens in eth_df.Tokens]
dict(eth_bigram_counts[0].most_common(10))

In [ ]:
# Use the token_count function to generate the top 10 words from each coin
def token_count(tokens, N=10):
    """Returns the top N tokens from the frequency count"""
    return Counter(tokens).most_common(N)

In [ ]:
# Get the top 10 words for Bitcoin
bigstr = []
for tokens in btc_df.Tokens:
    bigstr += tokens
btc_token_count = token_count(bigstr)
btc_token_count

In [ ]:
# Get the top 10 words for Ethereum
bigstr = []
for tokens in eth_df.Tokens:
    bigstr += tokens
eth_token_count = token_count(bigstr)
eth_token_count

# Word Clouds

In this section, you will generate word clouds for each coin to summarize the news for each coin

In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')
import matplotlib as mpl
mpl.rcParams['figure.figsize'] = [20.0, 10.0]

In [ ]:
# Generate the Bitcoin word cloud
btc_corpus=''
sw = set(stopwords.words('english') + expanded_stopwords)
for token in btc_df.Text:
    btc_corpus += token
long_string = " ".join(str(x) for x in tokenizer(btc_corpus))
wc = WordCloud(collocations=False).generate(long_string)
plt.title('Bitcoin Word Cloud', fontsize=20, fontweight='bold')
plt.style.use('seaborn-whitegrid')
plt.imshow(wc)
plt.axis("off")
plt.show()

In [ ]:
# Generate the Ethereum word cloud
eth_corpus=''
sw = set(stopwords.words('english') + expanded_stopwords)
for token in eth_df.Text:
    eth_corpus += token
long_string = " ".join(str(x) for x in tokenizer(eth_corpus))
wc = WordCloud(collocations=False).generate(long_string)
plt.title('Ethereum Word Cloud', fontsize=20, fontweight='bold')
plt.style.use('seaborn-whitegrid')
plt.imshow(wc)
plt.axis("off")
plt.show()

# Named Entity Recognition

In this section, you will build a named entity recognition model for both coins and visualize the tags using SpaCy.

In [ ]:
import spacy
from spacy import displacy

In [ ]:
# Optional - download a language model for SpaCy
# !python -m spacy download en_core_web_sm - DONE

In [ ]:
# Load the spaCy model
nlp = spacy.load('en_core_web_sm')

## Bitcoin NER

In [ ]:
# Concatenate all of the bitcoin text together
btc_corpus

In [ ]:
# Run the NER processor on all of the text
btc_ner = nlp(btc_corpus)
# Add a title to the document
btc_ner.user_data['title'] = "Bitcoin NER"

In [ ]:
# Render the visualization
displacy.render(btc_ner, style='ent')

In [ ]:
# List all Entities
btc_entities = set([ent.text for ent in btc_ner.ents ])
btc_entities

---

## Ethereum NER

In [ ]:
# Concatenate all of the bitcoin (eth?) text together
eth_corpus

In [ ]:
# Run the NER processor on all of the text
eth_ner = nlp(eth_corpus)
# Add a title to the document
eth_ner.user_data['title'] = "Ethereum NER"

In [ ]:
# Render the visualization
displacy.render(eth_ner, style='ent')

In [ ]:
# List all Entities
eth_entities = ([ent.text for ent in eth_ner.ents])
set(eth_entities)